# Dapi Database Access

In [ ]:
# Dapi installation
!pip uninstall dapi --yes

!pip install dapi --user --quiet

# Install the latest development version of dapi from GitHub
# !pip install git+https://github.com/DesignSafe-CI/dapi.git@dev --user --quiet

# Install editable local version of dapi
# !pip install -e ../

Found existing installation: dapi 1.0.0
Uninstalling dapi-1.0.0:
  Successfully uninstalled dapi-1.0.0
  Cloning https://github.com/DesignSafe-CI/dapi.git (to revision t3) to /tmp/pip-req-build-f6bbn80i
  Running command git clone --filter=blob:none --quiet https://github.com/DesignSafe-CI/dapi.git /tmp/pip-req-build-f6bbn80i
  Running command git checkout -b t3 --track origin/t3
  Switched to a new branch 't3'
  Branch 't3' set up to track remote branch 't3' from 'origin'.
  Resolved https://github.com/DesignSafe-CI/dapi.git to commit 6e85ea5b2b26fe7a82220db6260ac6c2124caaee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dapi: filename=dapi-1.0.0-py3-none-any.whl size=24062 sha256=cf0ff0df24ae1cb99cf56a627d1fdcd6d5d91e53a3c9eae6e99b8f3ab519713a
  Stored in directory: /tmp/pip-ephem-wheel-cache-58or_9hm/wheels/e1/79/c6/a9ec0b77e5349b798d57dca8ea213ee48626dea9e159e67fdc
Successful

In [2]:
# Cell: Imports
import os
from dapi import DSClient  # Import only the main client

# Import exceptions if needed for specific handling later
from dapi import FileOperationError, JobSubmissionError, JobMonitorError
import json
from datetime import datetime
from dataclasses import asdict
import pandas as pd

In [3]:
# Authenticate & Initialize Client
try:
    print("Initializing DSClient...")
    ds = DSClient()  # This sets up ds.db internally
    print("DSClient initialized.")
except Exception as e:
    print(f"Initialization failed: {e}")
    raise SystemExit("Stopping notebook due to client initialization failure.")

Initializing DSClient...
Authentication successful.
DatabaseAccessor initialized. Connections will be created on first access.
DSClient initialized.


In [4]:
# Create Convenience Variables for Databases (Optional but meets user request)
print("\nCreating database convenience variables...")
try:
    # These lines trigger the lazy initialization in DatabaseAccessor._get_db
    # if the specific database hasn't been accessed yet.
    ngl = ds.db.ngl
    vp = ds.db.vp
    eq = ds.db.eq
    print("Database variables (ngl, vp, eq) created.")
    # Note: Engines/pools are now created, but connections are only used during queries.
except Exception as e:
    print(f"Failed to create database convenience variables: {e}")
    # Decide if this is critical - maybe only some DBs are needed?
    # For now, we'll continue but note the failure.
    ngl, vp, eq = None, None, None  # Ensure they exist but are None


Creating database convenience variables...
First access to 'ngl', initializing DSDatabase...
Creating SQLAlchemy engine for database 'sjbrande_ngl_db' (ngl)...
Engine for 'ngl' created.
First access to 'vp', initializing DSDatabase...
Creating SQLAlchemy engine for database 'sjbrande_vpdb' (vp)...
Engine for 'vp' created.
First access to 'eq', initializing DSDatabase...
Creating SQLAlchemy engine for database 'post_earthquake_recovery' (eq)...
Engine for 'eq' created.
Database variables (ngl, vp, eq) created.


In [6]:
## Database Example Usage (Using convenience variables)
if ngl:  # Check if ngl variable was successfully created
    try:
        print("\nQuerying NGL Database using 'ngl' variable...")
        sql = "SELECT * FROM SITE LIMIT 5"
        # Use the convenience variable directly
        df_ngl = ngl.read_sql(sql, output_type="DataFrame")
        print("--- NGL Data (DataFrame) ---")
        print(df_ngl)
    except Exception as e:
        print(f"\nAn error occurred during NGL database query: {e}")
else:
    print("\nSkipping NGL query as 'ngl' variable is not available.")


Querying NGL Database using 'ngl' variable...
Executing query on 'ngl'...
--- NGL Data (DataFrame) ---
   SITE_ID            SITE_NAME   SITE_LAT    SITE_LON  \
0      147            Amagasaki  34.715560  135.400750   
1      148         Bonds Corner  32.693100 -115.338200   
2      149          Hachirogata  39.850000  140.017000   
3      150  Higashi-Kobe Bridge  34.710214  135.293345   
4      151   Hanshin Expressway  34.724834  135.301489   

                    SITE_GEOL  \
0                         Qal   
1  Qal, deep, Imperial Valley   
2                   Af (Fill)   
3                   Af (Fill)   
4                               

                                            SITE_REM  SITE_STAT  SITE_REVW  
0  Industrial site near Yomoga River.  Coordinate...          1          2  
1                                                             1          2  
2  Gingery indicates the geology near the strong ...          1          2  
3  A geologic map of the area (Geologica